# Recurrent neural network (RNN) from scratch

# What is a recurrent neural network?

A recurrent neural network (RNN), as opposed to a regular fully connected neural network (FCNN), has layers that are connected to themselves.

The difference might be clearer by first looking at an FCNN.

<img src="Figures/fcnn.svg" width="720px" alt="FCNN">

In an FCNN there are no connections between nodes in a single layer. For instance, $h_1^1$ is not connected to $h_2^1$. In addition, the input and output are always of a fixed length.

In an RNN, however, this is no longer the case. Nodes in the hidden layers are connected to themselves, represented by the curved lines in the figure below.

<img src="Figures/rnn.svg" width="400px">

Thus the output $\vec{h}$ from the hidden layer is fed back into the hidden layer. This recurrence makes RNNs useful when working with sequential data, as we can have input of variable length. This is more clear if we unfold the recurrent part of the network.

<img src="Figures/rnn_unfold.svg" width="720px">

# The mathematics of RNNs

## The RNN architecture

Consider some sequential input $X$ with $n$ features. Note that $X$ here is an array with two axes, since it contains $n$ features at each time step in the sequence. We will denote the input at a specific time step $t$ as
$$\vec{X}^{(t)} = \begin{pmatrix}
X^{(t)}_1 \\ \vdots \\ X^{(t)}_n
\end{pmatrix},$$
which is then an $n$-dimensional vector.

Next, consider an RNN with $L$ hidden layers, and an output layer with $m$ features. We will denote the output of the $l$'th hidden layer at time step $t$ as
$$\vec{h}_l^{(t)} = \begin{pmatrix}
h_{l, 1}^{(t)} \\ \vdots \\ h_{l, n_l}^{(t)}
\end{pmatrix},$$
with $n_l$ being the number of features in the $l$'th hidden layer. The output of the RNN at time step $t$ is denoted
$$\hat{\vec{y}}^{(t)} = \begin{pmatrix}
\hat{y}_1 \\ \vdots \\ \hat{y}_m,
\end{pmatrix}$$
where the hat is there to distinguish the RNN output $\hat{\vec{y}}^{(t)}$ from the target value, which is denoted $\vec{y}^{(t)}$.
The RNN will then look like this.

<img src="Figures/large_rnn.svg" width="720px">

## Forward propagation

In order to propagate forward through the network we need some weights and biases to connect the nodes. To simplify the notation going forward, we will consider the input layer to be the *zeroth layer*, and the output layer to be the *$L+1$'th layer*. We need each node to propagate to the node at the next layer (keeping the time step constant), and the next time step (keeping the layer constant), except for the input and output layers which do not connect to each other (as illustrated in the diagram above).

Let $W^{l,l+1}$ be the weight matrix and $\vec{b}^{l,l+1}$ the bias vector, both connecting nodes at the $l$'th layer to the $l+1$'th layer, keeping the time step constant. Next, let $W^{ll}$ be the weight matrix and $\vec{b}^{ll}$ the bias vector, both connecting nodes at subsequent time steps in the same layer. Also, let $\sigma_l$ be the activation function in the $l$'th layer. Lastly, define the weighted sum $\vec{z}_l^{(t)}$ at layer $l$ and time step $t$ such that the output of the node is the activation of that weighted sum, that is, such that $\vec{h}_l^{(t)} = \sigma_l (\vec{z}_l^{(t)})$.

Using these definitions the output from the first hidden layer at the first time step is then
$$ \vec{h}_1^{(1)} = \sigma_1 \left( \vec{z}_1^{(1)} \right), $$
with
$$ \vec{z}_1^{(1)} = W^{01} \vec{X}^{(1)} + \vec{b}^{01}.$$

At later time steps we will also need to consider the contribution from the previous time step. Hence for $t \geq 2$ we will define
$$\left( \vec{z}_1^{(t)} \right)_\text{layer} = W^{01} X^{(t)} + \vec{b}^{01}$$
$$\left( \vec{z}_1^{(t)} \right)_\text{time} = W^{11} \vec{h}_1^{(t-1)} + \vec{b}^{11},$$
such that $\left( \vec{z}_1^{(t)} \right)_\text{layer}$ is the contribution from the previous layer, and $\left( \vec{z}_1^{(t)} \right)_\text{time}$ is the contribution from the previous time step. We then have
$$\vec{z}_1^{(t)} = \left( \vec{z}_1^{(t)} \right)_\text{layer} + \left( \vec{z}_1^{(t)} \right)_\text{time},$$
and
$$\vec{h}_1^{(t)} = \sigma_1 \left( \vec{z}_1^{(t)} \right).$$

The expression is exactly the same for any hidden node, but for $l \geq 2$ we substitute $\vec{X}^{(t)}$ with $\vec{h}_{l-1}^{(t)}$. Thus for the $l$'th layer and $t$'th time step we have
$$ \left( \vec{z}_l^{(t)} \right)_{layer} = W^{l-1,l} \vec{h}_{l-1}^{(t)} + \vec{b}^{l-1,l} $$
and
$$ \left( \vec{z}_l^{(t)} \right)_{time} = W^{ll} \vec{h}_{l}^{(t-1)} + \vec{b}^{ll}, $$
that combine to give
$$ \vec{z}_l^{(t)} = \left( \vec{z}_l^{(t)} \right)_{layer} + \left( \vec{z}_l^{(t)} \right)_{time}, $$
which in turn results in
$$ \vec{h}_l^{(t)} = \sigma_l \left( \vec{z}_l^{(t)} \right). $$
This is also valid at the first time step by setting $\left( \vec{z}_l^{(1)} \right)_\text{time} = 0$.

The expression for the output layer is exactly the same as above, but with $\left( \vec{z}_l^{(t)} \right)_\text{time} = 0$. Thus we have
$$ \vec{z}_{L+1}^{(t)} = \left( \vec{z}_{L+1}^{(t)} \right)_\text{layer} = W^{L,L+1} \vec{h}_L^{(t)} + \vec{b}^{L,L+1} $$
and
$$ \hat{\vec{y}}^{(t)} = \sigma_{L+1} \left( \vec{z}_{L+1}^{(t)} \right) $$

The equations given for the forward propagation can seem a bit messy, so it is nice to have a more visual aid of what is going on. Here is a diagram of the complete RNN including the weights and biases relating the different nodes.

<img src="Figures/feed_forward.svg" width="720px">

And here is a weights and biases connected to a single arbitrary node. The green arrows represent input to the node, and the red arrows represent the output from the node.

<img src="Figures/feed_forward_node.svg" width=300px>

And here is the connections resulting in $\vec{h}_l^{(t)}$ in more detail.

<img src="Figures/activation.svg" width=500px>

## Backpropagation through time (BPTT)

Backpropagation in an RNN works by comparing the output of the network to some target output (just as in the regular neural network), and propagating backwards through both the layers and the *time sequence*. It is therefore commonly referred to as *backpropagation through time* (BPTT). We will now derive the necessary equations to perform BPTT.

We assume that we have propagated forward through the network, and have produced some output $\hat{\vec{y}}^{(t)}$. We want to compare this with some target output value $\vec{y}^{(t)}$, and will do so through a cost function $C \left(\hat{\vec{y}}, \vec{y} \right)$. We will denote the cost at a specific time step $t$ by $C^{(t)} = C^{(t)} \left(\hat{\vec{y}}^{(t)}, \vec{y}^{(t)} \right)$, and the overall cost of the network as $C$.

From the cost function at each time step, we want to compute the gradient with respect to each weight and bias, that is, we want to compute
$$
\frac{\partial C}{\partial W^{l_1 l_2}} \; \text{ and } \; \frac{\partial C}{\partial \vec{b}^{l_1 l_2}}
$$

We will do this one layer at a time, starting at the output layer, and propagating backwards through time in each layer. We assume that we know the gradient of the cost function with respect to the output $\frac{\partial C^{(t)}}{\partial \hat{\vec{y}}^{(t)}}$, and start by finding the gradient with respect to the output weights and biases $W^{L,L+1}$ and $\vec{b}^{L,L+1}$.

### Backpropagation through the output layer

First, we want to find the gradient with respect to $\vec{z}_{L+1}^{(t)}$. The derivative of $C$ with respect to some element $z_{L+1, i}^{(t)}$ of the weighted sum is given by

\begin{align*}
\frac{\partial C}{\partial z_{L+1,i}^{(t)}} &= \frac{\partial C^{(t)}}{\partial z_{L+1,i}^{(t)}}
\\[4ex]
&= \sum_{j=1}^m \frac{\partial C^{(t)}}{\partial \hat{y}_j^{(t)}}  \frac{\partial \hat{y}_j^{(t)}}{\partial z_{L+1,i}^{(t)}}
\\[4ex]
&= \sum_{j=1}^m \frac{\partial C^{(t)}}{\partial \hat{y}_j^{(t)}}  \sigma_{L+1}^\prime \left( z_{L+1,i}^{(t)} \right) \delta_{ij}
\\[4ex]
&= \frac{\partial C^{(t)}}{\partial \hat{y}_i^{(t)}}  \sigma_{L+1}^\prime \left( z_{L+1,i}^{(t)} \right)
\end{align*}

where $\delta_{ij}$ is the Kronecker delta
$\delta_{ij} = \begin{cases}
0, & i \neq j\\
1, & i = j
\end{cases}$, and $\sigma_{L+1}^\prime$ denotes the derivative of the activation function, which we will assume to be known.
we can write this expression more compactly in vector form as
$$
\frac{\partial C}{\partial \vec{z}_{L+1}^{(t)}} = \frac{\partial C^{(t)}}{\partial \hat{\vec{y}}^{(t)}} \odot \sigma_{L+1}^\prime \left( \vec{z}_{L+1}^{(t)} \right),
$$
where $\odot$ denotes the *Hadamard product*, an elementwise multiplication of two vectors/matrices of same size.

---

<u>**Note:**</u> Sometimes the derivatives are real numbers like $\frac{\partial C^{(t)}}{\partial z_{L+1,i}^{(t)}}$, sometimes they are vectors such as $\frac{\partial C^{(t)}}{\partial \vec{z}_{L+1}^{(t)}}$, and sometimes they are matrices. I have not included any explicit notation to explain when they are what, but will assume that this is understood implicitly. A general rule would be to look at whether the expression contains indices like $i,j,k,\ldots$ or not.

<u>**Another note:**</u> There are a lot of indices to keep track of, so to make the notation simpler to follow I will try to follow these rules consistently:
- $l$ = layer index (with $L$ being the final hidden layer). If I need several layer indices I will use $l_1,l_2,\ldots$.
- $(t)$ = time step index.
- $i,j,k$ = vector/matrix elements.
- $n$ = number of input features (length of $\vec{X}$).
- $m$ = number of output features (length of $\hat{\vec{y}}$).
- $n_1,n_2,\ldots$ = number of features in hidden layer number $1,2,\ldots$.

<u>**Third note:**</u> I will not always write the upper bound of summations explicitly, but will assume that this is understood implicitly. For instance, $\sum_j W^{l-1,l}_{ij} h_{l-1,j}$ should be understood to mean $\sum_{j=1}^{n_{l-1}} W^{l-1,l}_{ij} h_{l-1,j}$, such that it sums over all elements of $\vec{h}_{l-1}$.

---

The derivative with respect to the weighted sum will be used a lot during backpropagation, so we will give it its own notation
$$ \vec{\delta}_{L+1}^{(t)} \equiv \frac{\partial C^{(t)}}{\partial \vec{z}_{L+1}^{(t)}} = \frac{\partial C^{(t)}}{\partial \hat{\vec{y}}^{(t)}} \odot \sigma_{L+1}^\prime \left( \vec{z}_{L+1}^{(t)} \right).$$
$\delta_{L+1}^{(t)}$ has one index downstairs (denoting layer), and one index upstairs in parentheses (denoting time step), so don't mix it up with the Kronecker delta $\delta_{ij}$, which I will consistently write with two indices downstairs.

From the delta we can find the cost gradient with respect to the output bias.
Note that the same weights and biases occur several times in the RNN, so we have to sum over each contribution. The cost gradients with respect to the weights and biases in layer $l$ are denoted $\frac{\partial C}{\partial W^{l-1,l}}$, $\frac{\partial C}{\partial W^{ll}}$, $\frac{\partial C}{\partial \vec{b}^{l-1,l}}$ and $\frac{\partial C}{\partial \vec{b}^{ll}}$, and we will denote the contribution at time step $t$ as $\left(\frac{\partial C}{\partial W^{l-1,l}} \right)^{(t)}$, $\left( \frac{\partial C}{\partial W^{ll}} \right)^{(t)}$, $\left( \frac{\partial C}{\partial \vec{b}^{l-1,l}} \right)^{(t)}$ and $\left( \frac{\partial C}{\partial \vec{b}^{ll}} \right)^{(t)}$ such that $\frac{\partial C}{\partial W^{l-1,l}} = \sum_t \left( \frac{\partial C}{\partial W^{l-1,l}}\right)^{(t)}$ and so on.
Using this notation, the gradient with respect to the output bias becomes

\begin{align*}
\left( \frac{\partial C}{\partial b^{L,L+1}_i} \right)^{(t)} &= \sum_{j=1}^m \frac{\partial C}{\partial z_{L+1,j}^{(t)}} \frac{\partial z_{L+1,j}^{(t)}}{\partial b^{L,L+1}_i}
\\[4ex]
&= \sum_{j=1}^m \frac{\partial C}{\partial z_{L+1,j}^{(t)}} \frac{\partial}{\partial b^{L,L+1}_i} \left( \sum_k W^{L,L+1}_{jk} h_{L,k}^{(t)} + b^{L,L+1}_j \right)
\\[4ex]
&= \sum_{j=1}^m \frac{\partial C}{\partial z_{L+1,j}^{(t)}} \delta_{ij}
\\[4ex]
&= \frac{\partial C}{\partial z_{L+1,i}^{(t)}}
\\[4ex]
&= \delta_{L+1,i}^{(t)}.
\end{align*}

Thus on vector form we have
$$ \left( \frac{\partial C}{\partial \vec{b}^{L,L+1}} \right)^{(t)} = \vec{\delta}_{L+1}^{(t)},$$
and finally
$$\frac{\partial C}{\partial \vec{b}^{L,L+1}} = \sum_t \left( \frac{\partial C}{\partial \vec{b}^{L,L+1}} \right)^{(t)}$$

We can also compute the gradient with respect to the output weights

\begin{align*}
\left( \frac{\partial C}{W^{L,L+1}_{ij}} \right)^{(t)} &= \sum_{k_1=1}^m \frac{\partial C}{\partial z_{L+1,k_1}^{(t)}} \frac{\partial z_{L+1,k_1}^{(t)}}{\partial W^{L,L+1}_{ij}}
\\[4ex]
&= \sum_{k_1=1}^m \delta_{L+1,k_1}^{(t)} \frac{\partial}{\partial W^{L,L+1}_{ij}}
\left( \sum_{k_2} W^{L,L+1}_{k_1 k_2} h_{L,k_2}^{(t)} + b^{L,L+1}_{k_1} \right)
\\[4ex]
&= \sum_{k_1=1}^m \delta_{L+1,k_1}^{(t)} \sum_{k_2} h_{L,k_2}^{(t)} \delta_{i k_1} \delta_{j k_2}
\\[4ex]
&= \delta_{L+1,i}^{(t)} h_{L,j}^{(t)}
\\[4ex]
&= \left[ \vec{\delta}_{L+1}^{(t)} \left(\vec{h}_{L}^{(t)}\right)^T \right]_{ij}.
\end{align*}

Thus on vector form we have

$$ \left( \frac{\partial C}{W^{L,L+1}} \right)^{(t)} = \vec{\delta}_{L+1}^{(t)} \left(\vec{h}_{L}^{(t)}\right)^T, $$
and
$$\frac{\partial C}{W^{L,L+1}} = \sum_t \left( \frac{\partial C}{W^{L,L+1}} \right)^{(t)}.$$

Note that we here have an outer product between two vectors, which results in a matrix:

$$
\vec{\delta}_{L+1}^{(t)} \left(\vec{h}_{L}^{(t)}\right)^T
=
\begin{pmatrix}
\delta_{L+1,1}^{(t)} \\ \vdots \\ \delta_{L+1,m}^{(t)}
\end{pmatrix}
\begin{pmatrix}
h_{L,1}^{(t)} & \cdots & h_{L,n_L}^{(t)}
\end{pmatrix}
=
\begin{pmatrix}
\delta_{L+1,1}^{(t)} h_{L,1}^{(t)} & \cdots & \delta_{L+1,1}^{(t)} h_{L,n_L}^{(t)}
\\
\vdots & \ddots & \vdots
\\
\delta_{L+1,m}^{(t)} h_{L,1}^{(t)} & \cdots & \delta_{L+1,m}^{(t)} h_{L,n_L}^{(t)}
\end{pmatrix}
$$

Lastly, we need to compute the gradient with respect to the output from the previous layer $\frac{\partial C}{\partial \vec{h}_L^{(t)}}$, in order to continue backpropagating through previous layers. We find this in much the same way as we found the other gradients above.

\begin{align*}
\frac{\partial C}{\partial h_{L,i}^{(t)}} &= \sum_j \frac{\partial C}{z_{L+1,j}^{(t)}} \frac{\partial z_{L+1,j}^{(t)}}{\partial h_{L,i}^{(t)}}
\\[4ex]
&= \sum_j \delta_{L+1,j}^{(t)} \frac{\partial}{\partial h_{L,i}^{(t)}} \left( \sum_k W^{L,L+1}_{jk} h_{L,k}^{(t)} + b_j^{L,L+1} \right)
\\[4ex]
&= \sum_j \delta_{L+1,j}^{(t)} \sum_k W^{L,L+1}_{jk} \delta_{ik}
\\[4ex]
&= \sum_j \delta_{L+1,j}^{(t)} W^{L,L+1}_{ji}
\\[4ex]
&= \sum_j \left[ \left( W^{L,L+1} \right)^T \right]_{ij} \delta_{L+1,j}^{(t)}
\\[4ex]
&= \left[ \left(W^{L,L+1} \right)^T \vec{\delta}_{L+1}^{(t)} \right]_i
\end{align*}

And thus on vector form we have

$$\frac{\partial C}{\partial \vec{h}_L^{(t)}} = \left( W^{L,L+1} \right)^T \vec{\delta}_{L+1}^{(t)}$$

Here is a diagram showing the backpropagation through the output layer.

<img src="Figures/backprop_output.svg" width=720px>

### Backpropagation through arbitrary node

Consider some arbitrary node in the RNN with output $\vec{h}_l^{(t)}$. Assume you know the total gradient of the cost with respect to this output from the two suceeding nodes
$$
\frac{\partial C}{\partial \vec{h}_l^{(t)}} = \left( \frac{\partial C}{\partial \vec{h}_l^{(t)}} \right)_\text{layer} + \left( \frac{\partial C}{\partial \vec{h}_l^{(t)}} \right)_\text{time}.
$$
We now want to compute the gradients with respect to the weights and biases connecting the two previous nodes to this node, so that we can update these weights and biases when training the network, as well as the gradient with respect to the two previous nodes, so that we can continue backpropagation through the other nodes. The situation is illustrated in the diagram below. The blue arrows show the input gradient from the succeeding nodes, and the red arrows show the gradients we want to compute.

<img src="Figures/backprop_node.svg" width=720px>

The necessary gradients are derived in the same way as for the output layer, so I will simply state the results here. We get the following set of equations for backpropagating through a general node in the RNN.
\begin{align}
\delta_l^{(t)} &= \frac{\partial C}{\partial \vec{h}_l^{(t)}} \odot \sigma_l^\prime \left(\vec{z}_l^{(t)} \right)
\\[4ex]
\left( \frac{\partial C}{\partial \vec{b}^{l-1,l}} \right)^{(t)} = \left( \frac{\partial C}{\partial \vec{b}^{ll}} \right)^{(t)} &= \delta_l^{(t)}
\\[4ex]
\left( \frac{\partial C}{\partial W^{l-1,l}} \right)^{(t)} &= \delta_l^{(t)} \left( \vec{h}_{l-1}^{(t)} \right)^T
\\[4ex]
\left( \frac{\partial C}{\partial W^{ll}} \right)^{(t)} &= \delta_l^{(t)} \left( \vec{h}_l^{(t-1)} \right)^T
\\[4ex]
\frac{\partial C}{\partial \vec{h}_{l-1}^{(t)}} &= \left[ \left( W^{l-1,l} \right)^{(t)} \right]^T \delta_l^{(t)}
\\[4ex]
\frac{\partial C}{\partial \vec{h}_{l}^{(t-1)}} &= \left[ \left( W^{ll} \right)^{(t-1)} \right]^T \delta_l^{(t)},
\end{align}

and
\begin{align}
\frac{\partial C}{\partial \vec{b}^{l-1,l}} &= \sum_t \left( \frac{\partial C}{\partial \vec{b}^{l-1,l}} \right)^{(t)}
\\[4ex]
\frac{\partial C}{\partial \vec{b}^{ll}} &= \sum_t \left( \frac{\partial C}{\partial \vec{b}^{ll}} \right)^{(t)}
\\[4ex]
\frac{\partial C}{\partial W^{l-1,l}} &= \sum_t \left( \frac{\partial C}{\partial W^{l-1,l}} \right)^{(t)}
\\[4ex]
\frac{\partial C}{\partial W^{ll}} &= \sum_t \left( \frac{\partial C}{\partial W^{ll}} \right)^{(t)}.
\end{align}

With this method we can start with the nodes in the output layer, and propagate backwards. The necessary input to one node is the output from backpropagating through the previous node. Thus we can use the equations above recursively, layer by layer, to backpropagate through the entire network.

<img src="Figures/backprop_steps.svg" width=100%>

# The RNN code

Now that we have the mathematical framework, we can develop the code for the RNN.

## Functions

Before we start building a recurrent neural network, we need to define some functions. We need activation functions, cost functions and a way to differentiate these. We also need gradient descent schedulers to update our weights and biases when backpropagating. These functions are defined in this section.

### Activation functions

We want to be able to choose which activation function to use in different layers of the RNN. Here we define some activation functions that can be used by the network. If you have developed a regular fully connected neural network in FYS-STK4155 these functions will probably look very familiar, as they are pretty much copied from those lecture notes. The main difference is that I have used JAX instead of autograd for automatic differentiation. Due to the way JAX vectorizes, I have not gotten gradients with jax to work for softmax, but have included grad_softmax() as its own function.

In [79]:
import numpy as np
import jax.numpy as jnp

def identity(X):
    return X


def sigmoid(X):
    try:
        return 1.0 / (1 + jnp.exp(-X))
    except FloatingPointError:
        return jnp.where(X > jnp.zeros(X.shape), jnp.ones(X.shape), jnp.zeros(X.shape))


def softmax(X):
    X = X - np.max(X, axis=-1, keepdims=True)
    delta = 10e-10
    return np.exp(X) / (np.sum(np.exp(X), axis=-1, keepdims=True) + delta)


def grad_softmax(X):
    f = softmax(X)
    return f - f**2


def RELU(X):
    return jnp.where(X > jnp.zeros(X.shape), X, jnp.zeros(X.shape))


def LRELU(X):
    delta = 10e-4
    return jnp.where(X > jnp.zeros(X.shape), X, delta * X)


def tanh(X):
    return jnp.tanh(X)

### Cost functions

Next, we need to implement the cost functions. We include three cost functions here. The ordinary least squares (OLS) is used for regression problems, and the logistic regression and cross-entropy are used for classification problems.

In [80]:
def CostOLS(target):

    def func(X):
        return (1.0 / target.shape[0]) * jnp.sum((target - X) ** 2)

    return func


def CostLogReg(target):

    def func(X):

        return -(1.0 / target.shape[0]) * jnp.sum(
            (target * jnp.log(X + 10e-10)) + ((1 - target) * jnp.log(1 - X + 10e-10))
        )

    return func


def CostCrossEntropy(target):

    def func(X):
        return -(1.0 / target.size) * jnp.sum(target * jnp.log(X + 10e-10))

    return func

### Automatic differentiation

As mentioned above, we use JAX for automatic differentiation, which is done with the function *grad* in the JAX library. For *grad* to work on a function, it cannot use regular numpy, but must use *jax.numpy*, which is why we imported and used this when defining the activation and cost functions. JAX's numpy is only used for these functions, while we stick with regular numpy for everything else. 

The RELU and leaky RELU activation functions are not continuously differentiable, so we will handle these explicitly in our code.

In [81]:
from jax import grad

def derivate(func):
    if func.__name__ == "RELU":

        def func(X):
            return jnp.where(X > 0, 1, 0)

        return func

    elif func.__name__ == "LRELU":

        def func(X):
            delta = 10e-4
            return jnp.where(X > 0, 1, delta)

        return func

    else:
        return grad(func)


Note that the *grad* function is not, in itself, vectorized. This means that if we send in an array to a function that has been differentiated, JAX will treat this as a function with an array as input, not as a function treating each element individually. This is better understood with an example.

Consider the function $f(x)=x^2$, with derivative $f^\prime (x)=2x$. With JAX we get this with

In [82]:
def f(x):
    return x**2

df = grad(f)

This works if we input a scalar value

In [83]:
x = 2.0
print(df(x))

4.0


But if we try to input an array of values we get an error message.

In [84]:
x = np.linspace(0, 3, 10)

try:
    print(df(x))
except TypeError as msg:
    print(f"Error message: {msg}")

Error message: Gradient only defined for scalar-output functions. Output had shape: (10,).


This is because JAX does not treat each element in $x$ individually. To get around this we can use the *vmap* function in the JAX library, which vectorizes the function.

In [85]:
from jax import vmap
df = vmap(grad(f))
x = np.linspace(0, 3, 10)
print(df(x))

[0.        0.6666667 1.3333334 2.        2.6666667 3.3333333 4.
 4.6666665 5.3333335 6.       ]


Note that *vmap* only vectorizes along one dimension. So if the input array contains several axes, we have to apply *vmap* for each axis.

In [86]:
df = vmap(vmap(vmap(grad(f))))
x = np.linspace(0,3, 27).reshape((3,3,3))
print(df(x))

[[[0.         0.23076923 0.46153846]
  [0.6923077  0.9230769  1.1538461 ]
  [1.3846154  1.6153846  1.8461539 ]]

 [[2.0769231  2.3076923  2.5384614 ]
  [2.7692308  3.         3.2307692 ]
  [3.4615386  3.6923077  3.9230769 ]]

 [[4.1538463  4.3846154  4.6153846 ]
  [4.8461537  5.076923   5.3076925 ]
  [5.5384617  5.769231   6.        ]]]


This approach is tedious, yes, but as far as I know it is the only way to make JAX differentiate elementwise. If you find a simpler work-around, feel free to share with the professor or the group teachers so these notes can be updated.

### Schedulers

We also want to be able to choose which method we want to use for gradient descent when training the RNN. This is done by using a scheduler defined below. These schedulers are identical to the ones used in FYS-STK4155.

In [87]:
class Scheduler:
    """
    Abstract class for Schedulers
    """

    def __init__(self, eta):
        self.eta = eta

    # should be overwritten
    def update_change(self, gradient):
        raise NotImplementedError

    # overwritten if needed
    def reset(self):
        pass


class Constant(Scheduler):
    def __init__(self, eta):
        super().__init__(eta)

    def update_change(self, gradient):
        return self.eta * gradient

    def reset(self):
        pass


class Momentum(Scheduler):
    def __init__(self, eta: float, momentum: float):
        super().__init__(eta)
        self.momentum = momentum
        self.change = 0

    def update_change(self, gradient):
        self.change = self.momentum * self.change + self.eta * gradient
        return self.change

    def reset(self):
        pass


class Adagrad(Scheduler):
    def __init__(self, eta):
        super().__init__(eta)
        self.G_t = None

    def update_change(self, gradient):
        delta = 1e-8  # avoid division ny zero

        if self.G_t is None:
            self.G_t = np.zeros((gradient.shape[0], gradient.shape[0]))

        self.G_t += gradient @ gradient.T

        G_t_inverse = 1 / (
            delta + np.sqrt(np.reshape(np.diagonal(self.G_t), (self.G_t.shape[0], 1)))
        )
        return self.eta * gradient * G_t_inverse

    def reset(self):
        self.G_t = None


class AdagradMomentum(Scheduler):
    def __init__(self, eta, momentum):
        super().__init__(eta)
        self.G_t = None
        self.momentum = momentum
        self.change = 0

    def update_change(self, gradient):
        delta = 1e-8  # avoid division ny zero

        if self.G_t is None:
            self.G_t = np.zeros((gradient.shape[0], gradient.shape[0]))

        self.G_t += gradient @ gradient.T

        G_t_inverse = 1 / (
            delta + np.sqrt(np.reshape(np.diagonal(self.G_t), (self.G_t.shape[0], 1)))
        )
        self.change = self.change * self.momentum + self.eta * gradient * G_t_inverse
        return self.change

    def reset(self):
        self.G_t = None


class RMS_prop(Scheduler):
    def __init__(self, eta, rho):
        super().__init__(eta)
        self.rho = rho
        self.second = 0.0


    def update_change(self, gradient):
        delta = 1e-8  # avoid division ny zero
        self.second = self.rho * self.second + (1 - self.rho) * gradient * gradient
        return self.eta * gradient / (np.sqrt(self.second + delta))

    def reset(self):
        self.second = 0.0


class Adam(Scheduler):
    def __init__(self, eta, rho, rho2):
        super().__init__(eta)
        self.rho = rho
        self.rho2 = rho2
        self.moment = 0
        self.second = 0
        self.n_epochs = 1

    def update_change(self, gradient):
        delta = 1e-8  # avoid division ny zero
        self.moment = self.rho * self.moment + (1 - self.rho) * gradient
        self.second = self.rho2 * self.second + (1 - self.rho2) * gradient * gradient

        moment_corrected = self.moment / (1 - self.rho**self.n_epochs)
        second_corrected = self.second / (1 - self.rho2**self.n_epochs)

        return self.eta * moment_corrected / (np.sqrt(second_corrected + delta))

    def reset(self):
        self.n_epochs += 1
        self.moment = 0
        self.second = 0

## The RNN

We will now implement the code for the RNN. The network will be object-oriented, consisting of the following classes:

- *RNN*: The complete network, consisting of several *Layer* objects.
- *Layer*: Abstract class containing information that is shared across the different types of layers. It is the parent class of the following:
    - *InputLayer*: Layer containing the input to the network. Does not contain any weights and biases.
    - *RNNLayer*: The recurrent layer consisting of nodes in sequence.
    - *OutputLayer*: Recurrent layer for output. Similar to RNNLayer, but does not have any connections between the nodes, only to the nodes at the same time step in the previous layer.
    - *DenseLayer*: Fully connected layer, used to switch from a recurrent network to a regular fully connected network. Especially used for non-sequential output, for instance in classification where you want to classify the entire sequence with a single output.
- *Node*: Contains information about a single node. This is where all the math of forward- and backpropagation takes place.

In our code we want to be able to input batches of values, so that we can feed forward (and backpropagate) many inputs at once. The input to our network will therefore be an array with three axes:

- The input axis, separating the different inputs in the batch.
- The sequence axis, separating the different time steps of the sequence.
- The feature axis, separating the different features of the input (the vector elements).

This input is fed forward through the network, thus each layer has the same three axes. Within the layers, we separate each time step into their own node, thus the nodes only have two axes: the input and feature axes.

Since the RNN class is dependent on the Layer classes, and the Layer classes are dependent on the Node class, we will build the network from the down up, starting with the Node class.

### The Node class

The Node class takes care of the math discussed in the sections [Forward propagation](#forward-propagation) and [Backpropagation through arbitrary node](#backpropagation-through-arbitrary-node). I restate the relevant equations below.

<p align=center><u>Forward propagation:</u></p>

\begin{align*}
\left( \vec{z}_l^{(t)} \right)_{layer} &= W^{l-1,l} \vec{h}_{l-1}^{(t)} + \vec{b}^{l-1,l}
\\[4ex]
\left( \vec{z}_l^{(t)} \right)_{time} &= W^{ll} \vec{h}_{l}^{(t-1)} + \vec{b}^{ll}
\\[4ex]
\vec{z}_l^{(t)} &= \left( \vec{z}_l^{(t)} \right)_{layer} + \left( \vec{z}_l^{(t)} \right)_{time}
\\[4ex]
\vec{h}_l^{(t)} &= \sigma_l \left( \vec{z}_l^{(t)} \right)
\end{align*}

<p align=center><u>Backpropagation:</u></p>

\begin{align*}
\frac{\partial C}{\partial \vec{h}_l^{(t)}} &= \left( \frac{\partial C}{\partial \vec{h}_l^{(t)}} \right)_\text{layer} + \left( \frac{\partial C}{\partial \vec{h}_l^{(t)}} \right)_\text{time}
\\[4ex]
\delta_l^{(t)} &= \frac{\partial C}{\partial \vec{h}_l^{(t)}} \odot \sigma_l^\prime \left(\vec{z}_l^{(t)} \right)
\\[4ex]
\left( \frac{\partial C}{\partial \vec{b}^{l-1,l}} \right)^{(t)} = \left( \frac{\partial C}{\partial \vec{b}^{ll}} \right)^{(t)} &= \delta_l^{(t)}
\\[4ex]
\left( \frac{\partial C}{\partial W^{l-1,l}} \right)^{(t)} &= \delta_l^{(t)} \left( \vec{h}_{l-1}^{(t)} \right)^T
\\[4ex]
\left( \frac{\partial C}{\partial W^{ll}} \right)^{(t)} &= \delta_l^{(t)} \left( \vec{h}_l^{(t-1)} \right)^T
\\[4ex]
\frac{\partial C}{\partial \vec{h}_{l-1}^{(t)}} &= \left[ \left( W^{l-1,l} \right)^{(t)} \right]^T \delta_l^{(t)}
\\[4ex]
\frac{\partial C}{\partial \vec{h}_{l}^{(t-1)}} &= \left[ \left( W^{ll} \right)^{(t-1)} \right]^T \delta_l^{(t)}.
\end{align*}

---

<u>**Note:**</u> The input and output from a node are two-dimensional arrays, with different inputs along the first axis, and the features along the second axis. The equations we have derived for computing forward- and backpropagation assume that we are working with column vectors, but when performing matrix multiplication with these two-dimensional arrays we are in practice working with row vectors. This has to be taken into account in our code.

Consider, for instance, multiplying some weight matrix $W$ with a vector $\vec{h}$. The way we have derived the equations this will look like

$$
W\vec{h} = \begin{pmatrix}
W_{11} & \cdots & W_{1n}\\
\vdots & \ddots & \vdots\\
W_{m1} & \cdots & W_{mn}
\end{pmatrix} \begin{pmatrix}
h_1 \\ \vdots \\ h_n
\end{pmatrix}.
$$

In the code, however, this will look like

$$
\vec{h} W = \begin{pmatrix}
h_{11} & \cdots & h_{1n}\\
\vdots & \ddots & \vdots\\
h_{N1} & \cdots & h_{Nn}
\end{pmatrix} \begin{pmatrix}
W_{11} & \cdots & W_{1m}\\
\vdots & \ddots & \vdots\\
W_{n1} & \cdots & W_{nm}
\end{pmatrix},
$$

where $N$ is the number of inputs. Note that $W$ and $\vec{h}$ have switched places in the matrix multiplication, and that $W$ is essentially a transpose of the corresponding $W$ in the previous case (since we are multiplying the columns with the $\vec{h}$-vectors, instead of the rows).

This is not a big difference, but can be taken into account by reversing the order of some of the matrix multiplications. The full set of changes that must be done to account for this are listed here:

$$
\begin{align}
W \vec{h} \to \vec{h} W \\
\vec{\delta} \vec{h}^T \to \vec{h}^T \vec{\delta}\\
W^T \vec{\delta} \to \vec{\delta} W^T,
\end{align}
$$

and are taken care of in the code below.

---

In [88]:
from collections.abc import Callable # Used for type hints of functions

class Node:
    """
    Single node in the RNN. Computes forward propagation through a single node, stores the output of as a vector (1D array) of length 'n_features'. This class also computes the backpropagation through the node, and stores all the relevant gradients.

    Attributes
    ----------
    n_features (int)
        Number of features in this node (the length of the vector).
    act_func (Callable)
        The activation function of this node.
    {b/W}_{layer/time} (ndarray)
        Bias(b) or weight matrix (W) for computing the contribution to this node from the node in the previous layer/time step.
    h_{layer/time/output} (ndarray)
        The output of the node in the previous layer/time step, or the output of this node.
    z_output (ndarray)
        The weighted sum output of this node (the output before activation).
    grad_{b/W/h}_{layer/time} (ndarray)
        The gradient of the cost function with respect to the appropriate variable.
    """
    def __init__(
            self,
            n_features: int,
            act_func: Callable[[np.ndarray], np.ndarray] = identity,
            W_layer: np.ndarray = None,
            b_layer: np.ndarray = None,
            W_time: np.ndarray = None,
            b_time: np.ndarray = None
    ):
        """
        Constructor for Node objects.

        Parameters
        ----------
        n_features (int)
            Number of features in this node (the length of the vector).
        act_func (Callable)
            The activation function of this node.
        {b/W}_{layer/time} (ndarray)
            Bias(b) or weight matrix (W) for computing the contribution to this node from the node in the previous layer/time step.
        """
        self.n_features = n_features
        self.act_func = act_func
        self.W_layer = W_layer
        self.b_layer = b_layer
        self.W_time = W_time
        self.b_time = b_time

        ## Values from feed_forward()
        self.h_layer = None # h from previous layer
        self.h_time = None # h from previous time step
        self.z_output = None # z for this node
        self.h_output = None # h from this node

        ## Values from backpropagate()
        self.grad_b_layer = None
        self.grad_b_time = None
        self.grad_W_layer = None
        self.grad_W_time = None
        self.grad_h_layer = None
        self.grad_h_time = None
    
    def set_Wb(
            self,
            W_layer: np.ndarray,
            b_layer: np.ndarray,
            W_time: np.ndarray,
            b_time: np.ndarray
    ):
        """
        Sets the weights and biases to specific values. Used by the layer classes to ensure all nodes have correct weights and biases.

        Parameters
        ----------
        {b/W}_{layer/time} (ndarray)
            Bias(b) or weight matrix (W) for computing the contribution to this node from the node in the previous layer/time step.
        """
        self.W_layer = W_layer
        self.b_layer = b_layer
        self.W_time = W_time
        self.b_time = b_time

    def feed_forward(
            self,
            h_layer: np.ndarray,
            h_time: np.ndarray = None
    ) -> np.ndarray:
        """
        Computes the output of this node from the output of the nodes at the previous layer and time step, using the equations for forward propagation.

        Parameters
        ----------
        h_{layer/time/output} (ndarray)
            The output of the node in the previous layer/time step.
        
        Returns
        -------
        h_output (ndarray)
            The output of this node.
        """
        ## Save h_layer and h_time for use in backpropagation
        self.h_layer = h_layer
        self.h_time = h_time

        num_inputs = h_layer.shape[0]

        ## Compute weighted sum z for this node.
        z_layer = h_layer @ self.W_layer + self.b_layer # Dimension example: (100,5)@(5,7) + (7) = (100,7)

        if h_time is None:
            # This node is at the first time step, thus not receiving any input from previous time steps.
            z_time = np.zeros((num_inputs, self.n_features))
        else:
            z_time = h_time @ self.W_time + self.b_time
        
        self.z_output = z_layer + z_time # Save the weighted sum in the node

        ## Compute activation of the node
        h_output = self.act_func(self.z_output)

        self.h_output = h_output # Save the output in the node
        return h_output # Return output
        

    def backpropagate(
            self,
            dC_layer: np.ndarray = None,
            dC_time: np.ndarray = None,
            lmbd: float = 0.01
    ):
        """
        Performs backpropagation through this node. Computes the gradient of the cost function with respect to the weights and biases of this layer (and stores these gradients so the weights and biases can be updated in the layer object this node belongs to), and the gradient with respect to the output of the nodes in the previous layer and time step (and stores these as well for further backpropagation through the network).

        Parameters
        ----------
        dC_{layer/time} (ndarray)
            Contribution of cost gradient w.r.t. this node from node at next layer/time.
        lmbd (float)
            Regularization parameter for finding the cost gradient with respect to the weights.
        """
        n_batches = self.h_output.shape[0]
        
        ## Total gradient is the sum of the gradient from "next" layer and time
        if dC_time is None:
            # If this is the last node in the layer, the gradient is just the gradient from the next layer
            dC = dC_layer
        elif dC_layer is None:
            # If the next layer has no node at this time step (because it is a SingleOutputLayer), use only dC_time
            dC = dC_time
        else:
            dC = dC_layer + dC_time
        
        ## delta (gradient of cost w.r.t. z)
        if self.act_func.__name__ == "softmax":
            grad_act = grad_softmax(self.z_output)
        else:
            grad_act = vmap(vmap(derivate(self.act_func)))(self.z_output) # vmap is necessary for jax to vectorize gradient properly
        
        delta = grad_act * dC # Hadamard product, i.e., elementwise multiplication

        ## Gradients w.r.t. bias
        self.grad_b_layer = np.sum(delta, axis=0) / n_batches
        self.grad_b_time = np.sum(delta, axis=0) / n_batches

        ## Gradients w.r.t. weights
        self.grad_W_layer = self.h_layer.T @ delta / n_batches
        self.grad_W_layer = self.grad_W_layer + self.W_layer * lmbd # Regularization factor
        
        if self.h_time is None:
            self.grad_W_time = None
        else:
            self.grad_W_time = self.h_time.T @ delta / n_batches
            self.grad_W_time = self.grad_W_time + self.W_time * lmbd # Regularization factor

        ## Gradients w.r.t. input from previous nodes
        # Need to not transpose delta in order for matrices to match up correctly, since we have batches along rows, and features along columns
        self.grad_h_layer = delta @ self.W_layer.T
        if self.h_time is None:
            self.grad_h_time = None
        else:
            self.grad_h_time = delta @ self.W_time.T

### The Layer class

The RNN consists of several layers of various types. The Layer class is an abstract class keeping track of attributes and methods that are common for the different types of layers. We define this class for better organization of the different layers, but it should never be used by itself, for instance by creating a Layer object. You will notice that most of the Layer methods are only declared, not implemented. These methods will vary between layer types, and are implemented in the Layer's child classes, although the Layer class gives a nice overview of which methods to expect from its children.

In [111]:
from __future__ import annotations # Necessary to create typing hint of Layer within the class Layer

class Layer:
    """
    Abstract class for layers. The attributes given here are the attributes that are common to all layers, but they will also contain other attributes in addition to these.

    Attributes
    ----------
    n_features (int)
        Number of features in the nodes of this layer (the length of the vectors).
    seed (int)
        Seed for random number generating with numpy.
    nodes (list)
        List containing all the nodes of the layer in sequence, such that nodes[0] is the node at the first time step, nodes[1] at the second time step, and so on.
    n_nodes (int)
        Number of nodes in the layer. Is updated when adding or removing nodes.
    """
    def __init__(
            self,
            n_features: int,
            seed: int = 100
    ):
        """
        Constructor to be called from child classes.

        Parameters
        ----------
        n_features (int)
            Number of features in the nodes of this layer (the length of the vectors).
        seed (int)
            Seed for random number generating with numpy.
        """
        self.n_features = n_features
        self.seed = seed
        self.nodes = []
        self.n_nodes = 0
    
    def reset_weights(self):
        raise NotImplementedError
    
    def reset_schedulers(self):
        raise NotImplementedError
    
    def update_weights_all_nodes(self):
        raise NotImplementedError
    
    def add_node(self):
        raise NotImplementedError
    
    def remove_nodes(self):
        """
        Remove all the nodes created for this layer.

        NOTE
        ----
        The weights and biases of the nodes are still stored in the layer, so we can easily
        create new nodes. Removing the nodes is used to allow the sequence length to vary with
        each call of feed_forward().
        """
        self.nodes = []
        self.n_nodes = 0

    def feed_forward(
            self,
            prev_layer: Layer
    ):
        raise NotImplementedError
    
    def backpropagate(
            self,
            next_layer: Layer,
            lmbd: float
    ):
        raise NotImplementedError

### The InputLayer class

When we want to feed forward through an RNN, we will send a 3-dimensional array of shape (batch size, sequence length, number of features) as input. We then want to feed forward through the first hidden layer by computing the output of each node separately.
Each node should then get information from the node at the previous time step, and the node at the previous layer. The way we implement forward propagation in this notebook, we need the previous layer to contain one node for each time step, and feed forward through these nodes.
To make sure this also works for the first hidden layer, we include the InputLayer class, which takes the 3-dimensional input array to the network, and stores the value at each time step as 2-dimensional arrays in separate nodes. For all subsequent layers, we can then send in the previous layer as input to the feed_forward() method, work with each node separately.

Note that feed_forward() will take a layer class as input for all other layers, but since this is the first layer (with no preceding layer), it will instead take in the 3-dimensional input array.

In [113]:
class InputLayer(Layer):
    """
    The input layer of the RNN, used for storing the input to the RNN in nodes for easy forward propagation through subsequent layers.
    This class does not contain any weights or biases, nor does it implement backpropagation as there are no parameters to update.
    InputLayer is the only class that can function as the first layer in the RNN, and it should only be used for this purpose.

    Attributes
    ----------
    n_features (int)
        Number of features in the nodes of this layer (the length of the vectors).
    seed (int)
        Seed for random number generating with numpy.
    """
    def __init__(
            self,
            n_features: int,
            seed: int = 100
    ):
        """
        Constructor for InputLayer objects.

        Parameters
        ----------
        n_features (int)
            Number of features in the nodes of this layer (the length of the vectors).
        seed (int)
            Seed for random number generating with numpy.
        """
        super().__init__(n_features, seed)
    
    def add_node(self):
        """
        Add a node. Weights and biases are set to None, and activation to identity by default,
        as none of these are relevant for the input layer.
        """
        new_node = Node(self.n_features) # Activation and weights are not used for the input layer
        self.nodes.append(new_node)
        self.n_nodes += 1

    def feed_forward(
            self,
            X: np.ndarray
    ):
        """
        Feed forward through the layer. As this is the input layer, this amounts to adding nodes and setting the output of each node to the value of the input at the corresponding time step. Also checks that the number of features of the input is the same as the layer expects it to be.

        Parameters
        ----------
        X (ndarray)
            Input to the RNN, and thus to this layer. X has shape (batch size, sequence length, number of features)
        
        NOTE
        ----
        Unlike the other layers, this layer takes a numpy array as input instead of a Layer.
        """
        X_shape = X.shape
        sequence_length = X_shape[1]
        
        if not self.n_features == X_shape[2]:
            # Input must have the same number of features as defined by the layer
            raise ValueError(f"Expected the number of features in the input layer to be {self.n_features}, got {X_shape[2]}.")
        
        # Add a node to the layer for each time step, and set output
        self.remove_nodes()
        for i in range(sequence_length):
            self.add_node()
            self.nodes[i].h_output = X[:,i,:]